In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
# to use a specific GPU
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [3]:
import sys
sys.path.append('..')

In [4]:
from mgtbench.loading.dataloader import load_attribution_topic
# Humanities Social_sciences
data_dict = load_attribution_topic('Social_sciences')
y_train = data_dict["train"]['label']
X_train = data_dict["train"]['text']
y_test = data_dict["test"]['label']
X_test = data_dict["test"]['text']
import random


In [5]:
from mgtbench import AutoDetector, AutoExperiment
nclass = len(set(data_dict['train']['label']))
model= '/data1/zzy/roberta-base'
metric = AutoDetector.from_detector_name('LM-D',
                                        model_name_or_path=model,
                                        num_labels=nclass,
                                        use_metric=True
                                        )
experiment = AutoExperiment.from_experiment_name('supervised',detector=[metric])
experiment.load_data(data_dict)

config = {
    'need_finetune': True,
    'need_save': False,
    'epochs': 2,
    'batch_size': 64,
    'lr':5e-6,
    'disable_tqdm': True
    }
res = experiment.launch(**config)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /data1/zzy/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


length is set to 512
Calculate result for each data point
Running prediction of detector LM-D
False
using metric trainer


/home/lyl/miniconda3/envs/llm/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/lyl/miniconda3/envs/llm/lib/python3.10/site-packages/accelerate/accelerator.py:457: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the ke

Step,Training Loss
30,5.938900
60,4.939800
90,4.555600
120,4.454800
150,4.242400
180,4.161800
210,4.142500
240,4.057500
270,3.957400


Fine-tune finished
Predict training data
Predict testing data
Run classification for results
[[358   0   4   1   1   0]
 [  0 253   9  42  37  15]
 [  5   7 281  25   9  24]
 [  0  35  26 266  28   3]
 [  2  55   8  27 224  34]
 [  0  41  24   4  39 258]]
[[1406    0   16    1    1    0]
 [   2 1031   26  158  157   58]
 [  37   24 1180   67   55   74]
 [   3  134   60 1042  162   29]
 [   5  195   63  140  911  124]
 [   8  125   83   27  140 1039]]


In [6]:
res

[DetectOutput(name='test_pred', train=None, test=Metric(acc=0.7645687645687645, precision=0.7660343494346146, recall=0.7645687645687645, f1=0.7647463869086015, auc=-1.0, conf_m=array([[358,   0,   4,   1,   1,   0],
        [  0, 253,   9,  42,  37,  15],
        [  5,   7, 281,  25,   9,  24],
        [  0,  35,  26, 266,  28,   3],
        [  2,  55,   8,  27, 224,  34],
        [  0,  41,  24,   4,  39, 258]]))),
 DetectOutput(name='train_pred', train=None, test=Metric(acc=0.7700104858441105, precision=0.7700292688758228, recall=0.7700104858441105, f1=0.769749548423765, auc=-1.0, conf_m=array([[1406,    0,   16,    1,    1,    0],
        [   2, 1031,   26,  158,  157,   58],
        [  37,   24, 1180,   67,   55,   74],
        [   3,  134,   60, 1042,  162,   29],
        [   5,  195,   63,  140,  911,  124],
        [   8,  125,   83,   27,  140, 1039]])))]